# Self-Ask Exploration

The goal is to test out self-ask on complex questions from QAMPARI and RoMQA.

We'll start with QAMPARI, getting all of the complex questions from the dev set.

In [ ]:
import jsonlines

**Load & Sort QAMPARI Data**

In [ ]:
qmp_dev_data_path = "/scratch/ddr8143/multiqa/downloads/data/qampari/dev_data.jsonl"

In [ ]:
qmp_comp_devd = []
with open(qmp_dev_data_path) as f:
    qmp_devd_iter = jsonlines.Reader(f)
    for d in qmp_devd_iter:
        if 'wikitables' in d['qid'] or 'wikidata_simple' in d['qid']:
            continue
        qmp_comp_devd.append(d)

In [ ]:
print("Num Dev Non-Simple Wikidata Qs:", len(qmp_comp_devd))

In [ ]:
for i in range(30):
    print(i, qmp_comp_devd[i]['question_text'])

**Directly Use 4-shot Self-Ask Prompt**

In [ ]:
# Taken from: https://github.com/ofirpress/self-ask/blob/main/self-ask_plus_search-engine_demo.ipynb
base_prompt = ['''Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali 

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball 

Question: Are both the directors of Jaws and Casino Royale from the same country? 
Are follow up questions needed here: Yes. 
Follow up: Who is the director of Jaws? 
Intermediate Answer: The director of Jaws is Steven Spielberg. 
Follow up: Where is Steven Spielberg from? 
Intermediate Answer: The United States. 
Follow up: Who is the director of Casino Royale? 
Intermediate Answer: The director of Casino Royale is Martin Campbell. 
Follow up: Where is Martin Campbell from? 
Intermediate Answer: New Zealand. 
So the final answer is: No

Question: ''', 
'''
Are follow up questions needed here:''', ]

In [ ]:
# Also taken from: https://github.com/ofirpress/self-ask/blob/main/self-ask_plus_search-engine_demo.ipynb
# But then modified

#def promptf(question, prompt, intermediate = "\nIntermediate answer:", followup = "Follow up:", finalans= '\nSo the final answer is:'):
INTERMEDIATE = "\nIntermediate answer:"
FOLLOWUP = "Follow up:"
FINALANS = "\nSo the final answer is:"
def printprompt(qid, devqs, prompt):
    question = devqs[qid]['question_text']
    cur_prompt = prompt[0] +  question + prompt[1]

    print(cur_prompt, end ='')

    """
    ret_text = call_gpt(cur_prompt, intermediate)

    while followup in get_last_line(ret_text):

      
      cur_prompt += ret_text
      question = extract_question(ret_text)
      external_answer = get_answer(question)

      if external_answer is not None:
        cur_prompt += intermediate + ' ' + external_answer + '.'
        print(intermediate + ' ' + yellowfy(external_answer) + '.', end='' )
        ret_text = call_gpt(cur_prompt, intermediate)
      else:
        #We only get here in the very rare case that Google returns no answer.
        cur_prompt += intermediate
        print(intermediate + ' ')
        gpt_answer = call_gpt(cur_prompt, ['\n'+followup, finalans])
        cur_prompt += gpt_answer

    
    if finalans not in ret_text:
      cur_prompt += finalans
      print(finalans, end = '')
      ret_text = call_gpt(cur_prompt, '\n')

    return cur_prompt + ret_text
    """

In [ ]:
printprompt(0, qmp_comp_devd, base_prompt)

**Now Choose Subset to Try and Try Them**

In [ ]:
for i in range(30):
    offset = 200
    if "wikidata_comp" not in qmp_comp_devd[i+offset]['qid']:
        continue
    print(i+offset, qmp_comp_devd[i+offset]['question_text'])

In [ ]:
inds_to_try = [
    0,  # 0. screenwriter and director
    3,  # 1. where taught
    10, # 2. competed and won
    27, # 3. film steinbeck wrote
    32, # 4. institiution where educated
    44, # 5. graduated from two places
    59, # 6. which one did peerson win
    232,# 7. company produced wirtten
    255,# 8. composer for movie produced
    347,# 9. objects person designed depicted what
]

In [ ]:
for i in inds_to_try:
    print(qmp_comp_devd[i]['question_text'])

In [ ]:
test_ind = 0
print(qmp_comp_devd[inds_to_try[test_ind]]['question_text'])
print([a['answer_text'] for a in qmp_comp_devd[inds_to_try[test_ind]]['answer_list']])

print()

printprompt(inds_to_try[0], qmp_comp_devd, base_prompt)

**Make a better prompt**

In [ ]:
# Can we make a prompt out of these?
for i in [25, 95, 155, 385]:
    print(qmp_comp_devd[i]['question_text'])
    print([a['answer_text'] for a in qmp_comp_devd[i]['answer_list']])
    print()

In [ ]:
"""
[{
        'question_text': '',
        'question_type': '',
        'subquestions': [
            '',
            '',
        ]
    }],
"""
decomposed_qs_intersection = {
    0: [{
        'question_text': 'Harmony Korine was both screenwriter and director of what movie?',
        'question_type': 'intersection',
        'subquestions': [
            'Harmony Korine was the screenwriter of what movie?',
            'Harmony Korine was the director of what movie?',
        ]
    }],
    22: [{ # intersection
        "question_text": "Who was both a graduate from Ananda College and University of Ceylon?",
        "question_type": "intersection",
        "subquestions": [
            "Who graduated from Ananda College?",
            "Who graduated from University of Ceylon?",
        ],
    }],
    25: [{
        'question_text': 'Which movie had K. S. L. Swamy as its director and Vijaya Bhaskar as its musical composer?',
        'question_type': 'intersection',
        'subquestions': [
            'Which movie has K. S. L. Swamy as its director?',
            'Which movie has Vijaya Bhaskar as its musical composer?',
        ]
    }],
    90: [{ # intersection (but I see filter too)
        "question_text": "What Superbowls did the Washington Football Team play in?",
        "question_type": "intersection",
        "subquestions": [
            "Which Superbowls were played?",
            "Which games did the Washington Football Team play in?"
        ]
    }],
    95: [{
        'question_text': 'What music was composed by Devi Sri Prasad and produced by Dil Raju?',
        'question_type': 'intersection',
        'subquestions': [
            'What music was composed by Devi Sri Prasad?',
            'What music was produced by Dil Raju?',
        ]
    }],
    380: [{ # intersection
        "question_text": "Which competition had Vitória F.C. and S.L. Benfica as participants?",
        "question_type": "intersection",
        "subquestions": [
            "Which competition had Vitória F.C. as a participant?",
            "Which competition had S.L. Benfica as a participant?",

        ],
    }],
    385: [{
        'question_text': 'What movie did Irwin Allen both direct and produce?',
        'question_type': 'intersection',
        'subquestions': [
            'What movie did Irwin Allen direct?',
            'What movie did Irwin Allen produce?',
        ]
    }],
}

"""
[{
        'question_text': '',
        'question_type': '',
        'subquestions': [
            '',
            '',
        ]
    }],
"""
decomposed_qs_composition = {
    11: [{
        'question_text': 'Joe Pasternak produced a motion picture that was directed by who?',
        'question_type': 'composition',
        'subquestions': [
            'What motion pictures did Joe Pasternak produce?',
            'Who directed it?',
        ]
    }],
    110: [{
        'question_text': 'Who directed the TV show that Tom Kauffman worked on as a screenwriter?',
        'question_type': 'composition',
        'subquestions': [
            'What TV shows did Tom Kauffman work on as a screenwriter?',
            'Who directed it?',
        ]
    }],
    120: [{
        'question_text': 'Where did former Bishops of Warrington go to school?',
        'question_type': 'composition',
        'subquestions': [
            'Who were the former Bishops of Warrington?',
            'Where did they go to school?',
        ]
    }],
    150: [{ # compositional
        "question_text": "Who was credited as director for a movie penned by Peter Baynham?",
        "question_type": "composition",
        "subquestions": [
            "What movies were penned by Peter Baynham?",
            "Who was credited the director for it?"
        ],
    }],
    221: [{
        'question_text': 'Where did an employee of University of Pennsylvania Law School receive their education?',
        'question_type': 'composition',
        'subquestions': [
            'Who was an employee of University of Pennsylvania Law School?',
            'Where did they receive their education?',
        ]
    }],
}

# many of these are intersections that are better answered as filters
decomposed_qs_filter = {
    90 : [{ # intersection (but I see filter too)
        "question_text": "What Superbowls did the Washington Football Team play in?",
        "question_type": "filter",
        "subquestions": [
            "Which Superbowls have been played?",
            "Did the Washington Football Team play in it?"
        ]
    }],
    155: [{
        'question_text': 'Which FA Cup Final featured the Tottenham Hotspurs as competitors?',
        'question_type': 'filter',
        'subquestions': [
            'Which FA Cup Finals were played?',
            'Did it feature the Tottenham Hotsurs as a competitor?',
        ]
    }],    
}

In [ ]:
qmp_comp_devd[155]['qid']

In [ ]:
# Then, lets focus on pure intersections for now as the first trial q is a pure intersection
"""
Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali 
"""

def qdata_to_prompt(qdata, qdecompose):
    #Question 2: {subqs2}""".format(
    return """
Question: {init_q}
Can this be decomposed: Yes.
Is this a composition or intersection question: {qtype}
Question 1: {subqs1}
Question 2: {subqs2}
So the final answers are: {answer_list}""".format(
        init_q=qdata['question_text'],
        qtype=qdecompose['question_type'],
        subqs1=qdecompose['subquestions'][0],
        subqs2=qdecompose['subquestions'][1],
        answer_list=", ".join([a['answer_text'] for a in qdata['answer_list']]),
    )

In [ ]:
qind = 22
print(qdata_to_prompt(qmp_comp_devd[qind], decomposed_qs_intersection[qind][0]))

In [ ]:
# Intersection Prompt
test_qind = 385
for qind in [
    0, 
    22, 
    25, 
    95,
    385
]:
    if qind == test_qind:
        continue
    print(qdata_to_prompt(qmp_comp_devd[qind], decomposed_qs_intersection[qind][0]))
print()
print("Question: " + qmp_comp_devd[test_qind]['question_text'])
print("Can this be decomposed:")

In [ ]:
test_qind = 11
for qind in [
    11, 
    110, 
    120, 
    150,
    221
]:
    if qind == test_qind:
        continue
    print(qdata_to_prompt(qmp_comp_devd[qind], decomposed_qs_composition[qind][0]))
print()
print("Question: " + qmp_comp_devd[test_qind]['question_text'])
print("Can this be decomposed:")

In [ ]:
# Combined: get q type too
test_qind = 25
curr_qs_comp = {**decomposed_qs_composition, **decomposed_qs_intersection}
for qind in [
    0, 
    11, 
    110, 
    22, 
    120, 
    25, 
]:
    if qind == test_qind:
        continue
    print(qdata_to_prompt(qmp_comp_devd[qind], curr_qs_comp[qind][0]))
print()
print("Question: " + qmp_comp_devd[test_qind]['question_text'])
print("Can this be decomposed:")

In [ ]:
test_qind = 221
for qind in [
    11, 
    110, 
    120, 
    150,
    221
]:
    if qind == test_qind:
        continue
    print(qdata_to_prompt(qmp_comp_devd[qind], decomposed_qs_composition[qind][0]))
print()
print("Question: " + qmp_comp_devd[test_qind]['question_text'])
print("Can this be decomposed:")

In [ ]:
def comp_ans(gta_str, preda_str):
    gta = list(set([a.replace(' ', '').lower() for a in gta_str.split(", ")]))
    preda = set(list([a.replace(' ', '').lower() for a in preda_str.split(", ")]))
    print("Num pred answers:", len(preda))
    num_in = 0
    for a in gta:
        if a in preda:
            num_in += 1
            
    print(f"Num GT pred: {num_in} / {len(gta)}")
    print(f"Num pred not GT: {len(preda) - num_in}")

In [ ]:
comp_ans(
    gta_str="University of Pennsylvania, University of Oklahoma, New College of Florida, University of Pennsylvania Law School, University of Toronto, Alfred University, Southwestern College, University of Pennsylvania, Stanford Law School, Merton College, St Antony's College, Harvard Law School, Harvard University, Yale University, Reading High School, Smith College, Harvard University, Yale Law School, University of Pennsylvania Law School, University of Chicago Law School, University of Pennsylvania Law School, Lower Merion High School, Massachusetts Institute of Technology, Islamic Azad University, University of Pennsylvania Law School, Northwestern University School of Law, Harvard University, University of California, Harvard Law School, University of North Carolina at Chapel Hill, University of Chicago, Yale Law School, University of Pennsylvania Law School, Yale Law School, Princeton University, Somerville College, Cornell University, University of Oklahoma, University of Michigan Law School, University of Pennsylvania Law School, University of Michigan, University of Pennsylvania Law School, University of Pennsylvania Law School",
    preda_str="Stanford University, Harvard University, Yale University, University of California, Berkeley, University of Virginia, Columbia University, University of Michigan, Duke University, University of Pennsylvania, Georgetown University, Cornell University, Northwestern University",
)